## **Software de cálculo numérico para treliças planas**

**Introdução**

**Explicação da estrutura do software**

In [15]:
#Importando bibliotecas necessárias
import matplotlib as mpl
import matplotlib.pyplot as plt
from math import *
import numpy as np
import xlrd

*Explicando nomes de variáveis da tabela*

<small>A funcao 'importa' retorna o numero de nos [nn], a matriz dos nos [N], o numero
de membros [nm], a matriz de incidencia [Inc], o numero de cargas [nc], o vetor
carregamento [F], o numero de restricoes [nr] e o vetor de restricoes [R] 
contidos no arquivo de entrada.</small>

MUDAR ISSO DEPOIS SO COPIEI E COLEI DE LA

In [16]:
#Importando as funções do arquivo funcoesTermosol.py e extraindo variáveis do arquivo de entrada
from funcoesTermosol import plota

from funcoesTermosol import importa
[nn,N,nm,Inc,nc,F,nr,R] = importa('entrada.xls')

from funcoesTermosol import geraSaida

In [24]:
#Visualizando os dados obtidos através do arquivo de entrada
Inc[:,1]
Inc[:,2]
Inc
R[0,0]


0.0

In [18]:
#Montando a matriz de conectividade
m_conectividade = np.zeros((nm,7))

#preencher as incidências automaticamente
m_conectividade[:,1] = Inc[:,0]
m_conectividade[:,2] = Inc[:,1]
m_conectividade[:,4] = Inc[:,3]

for c in range(nm):
    m_conectividade[c,0] = c + 1

    #cálculo do comprimento
    x1_calculo_L = N[0,int(m_conectividade[c,1]-1)]
    y1_calculo_L = N[1,int(m_conectividade[c,1]-1)]
    x2_calculo_L = N[0,int(m_conectividade[c,2]-1)]
    y2_calculo_L = N[1,int(m_conectividade[c,2]-1)]

    m_conectividade[c,3] = sqrt((x2_calculo_L - x1_calculo_L)**2 + (y2_calculo_L - y1_calculo_L)**2)

    m_conectividade[c,6] = (y2_calculo_L - y1_calculo_L)/m_conectividade[c,3]
    m_conectividade[c,5] = (x2_calculo_L - x1_calculo_L)/m_conectividade[c,3]
    


m_conectividade

array([[ 1.e+00,  1.e+00,  2.e+00,  4.e-01,  2.e-04,  0.e+00,  1.e+00],
       [ 2.e+00,  2.e+00,  3.e+00,  3.e-01,  2.e-04,  1.e+00,  0.e+00],
       [ 3.e+00,  3.e+00,  1.e+00,  5.e-01,  2.e-04, -6.e-01, -8.e-01]])

In [19]:
#Cálculo da matriz de rigidez de cada elemento e global

m_rigidez_global = np.zeros((nn*2,nn*2))

for i in range(nm):
    multiplicador = (Inc[i,2] * Inc[i,3])/m_conectividade[i,3]
    
    s = m_conectividade[i,6]
  
    c = m_conectividade[i,5]

    m_rigidez_elemento = np.zeros((4,4))
    m_rigidez_elemento =   multiplicador * np.array([
        [c**2, c * s, -c**2, -c * s],
        [c * s, s**2, -c * s, -s**2],
        [-c**2, -c * s, c**2, c * s],
        [-c * s, -s**2, c * s, s**2]
    ])

    no_i_elemento = m_conectividade[i,1]
    no_j_elemento = m_conectividade[i,2]

    #atraves dos nós de i e j da matriz (POR IMAGEM P EXPLICAR MELHOR) calculo da posição na matriz global, a subtração vem de uma convenção
    index_i = np.array([2*no_i_elemento - 2, 2*no_i_elemento - 1], dtype=int)
    index_j = np.array([2*no_j_elemento - 2, 2*no_j_elemento - 1], dtype=int)

    #agora precisamos contemplar todas as possibilidades de combinação dos nós incidentes, então a contribuição que ele faz ao nó incidente em si (linhas 1 e 4 a seguir) e a contribuição que ele faz aos dois nós simultaneamente(linhas 2 e 3 a seguir), além disso usamos a função np.ix_ para coonseguirmos abarcar o intervalo de indices desejado 
    m_rigidez_global[np.ix_(index_i,index_i)] += m_rigidez_elemento[:2, :2]
    m_rigidez_global[np.ix_(index_i,index_j)] += m_rigidez_elemento[:2, 2:]
    m_rigidez_global[np.ix_(index_j,index_i)] += m_rigidez_elemento[2:, :2]
    m_rigidez_global[np.ix_(index_j,index_j)] += m_rigidez_elemento[2:, 2:]


m_rigidez_global


array([[ 3.0240e+07,  4.0320e+07,  0.0000e+00,  0.0000e+00, -3.0240e+07,
        -4.0320e+07],
       [ 4.0320e+07,  1.5876e+08,  0.0000e+00, -1.0500e+08, -4.0320e+07,
        -5.3760e+07],
       [ 0.0000e+00,  0.0000e+00,  1.4000e+08,  0.0000e+00, -1.4000e+08,
         0.0000e+00],
       [ 0.0000e+00, -1.0500e+08,  0.0000e+00,  1.0500e+08,  0.0000e+00,
         0.0000e+00],
       [-3.0240e+07, -4.0320e+07, -1.4000e+08,  0.0000e+00,  1.7024e+08,
         4.0320e+07],
       [-4.0320e+07, -5.3760e+07,  0.0000e+00,  0.0000e+00,  4.0320e+07,
         5.3760e+07]])

In [30]:
#montando a matriz de forças global e aplicando as restrições

m_rigidez_contorno = m_rigidez_global.copy()

m_rigidez_contorno[np.array(R, dtype=int), :] = 0
m_rigidez_contorno[:, np.array(R, dtype=int)] = 0
m_rigidez_contorno[np.array(R, dtype=int),np.array(R, dtype=int)] = 1

gdl_nrestritos = np.setdiff1d(np.arange(nn * 2),np.array(R, dtype = int))

m_rigidez_contorno = m_rigidez_contorno[gdl_nrestritos][:,gdl_nrestritos]
F_contorno = F[gdl_nrestritos]


array([[   0.],
       [ 150.],
       [-100.]])